In [1]:
import torch
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )

import os
import torch
from d2l import torch as d2l
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw, MolFromSmiles, MolToSmiles
import json

import utils


import json
import numpy as np
import torch

from con_function import tokenize, new_lines_ms, truncate_pad, load_corpus_ms
from utils_plot import plot_token_frequency, plot_bigram

with open('data/mass_spectra.json') as f_ms:
    mass_spectras = json.load(f_ms)
print(len(mass_spectras))



new_mass_spectras = [list(map(float, mass_spectra.split())) for mass_spectra in mass_spectras]
length_new_mass_spectras = [len(list(map(float, mass_spectra.split()))) for mass_spectra in mass_spectras]
# 有几个未取到数值？？？
# print(f"max_length_mass_spectras: {max(length_new_mass_spectras)}, min_length_mass_spectras: {min(length_new_mass_spectras)}")
print(f"max_length_mass_spectras: {max(length_new_mass_spectras)}")


len_new_mass_spectras = list(map(len, new_mass_spectras))
sub = np.array(len_new_mass_spectras)<=100
print(f"number of len_new_mass_spectras<=100: {np.sum(sub)}")
print(f"percent of total: {np.sum(sub)/len(new_mass_spectras)*100}")
print(f"set(len_new_mass_spectras): {set(len_new_mass_spectras)}")

# 绘制new_mass_spectras的长度分布情况
plot_token_frequency(seq_lengths=len_new_mass_spectras, name='new_mass_spectras')
 
new_mass_spectras_tokens = new_lines_ms(new_mass_spectras)
print(f"new_mass_spectras_tokens: {new_mass_spectras_tokens[0]}")
# corpus: 索引号     vocab：所有元素
src_corpus, src_vocab = load_corpus_ms(new_mass_spectras, max_tokens=-1)
print(f"len(src_corpus): {len(src_corpus)} \tlen(src_vocab): {len(src_vocab)}")


print(src_vocab['<unk>'], src_vocab['<pad>'], src_vocab['<bos>'], src_vocab['<eos>'])

# for i in range(1):
#     print(f"\t{truncate_pad(src_vocab[new_mass_spectras_tokens[i]], max(length_new_mass_spectras), src_vocab['<pad>'])}")

numpy_mass_spectra = [truncate_pad(src_vocab[token], max(length_new_mass_spectras), src_vocab['<pad>']) for token in new_mass_spectras_tokens]
print(len(numpy_mass_spectra), len(new_mass_spectras_tokens))
new_mass_spectras_tokens = list(new_mass_spectras_tokens) 
for i in np.arange(len(new_mass_spectras_tokens)):
    for j in np.arange(len(new_mass_spectras_tokens[i])):
        numpy_mass_spectra[i][j] = new_mass_spectras_tokens[i][j]
# print(numpy_mass_spectra.shape)

numpy_mass_spectra = numpy_mass_spectra[:][:100]
tensor_mass_spectra = torch.tensor(numpy_mass_spectra)

print(tensor_mass_spectra[0])


src_num_steps, tgt_num_steps = max(length_new_mass_spectras), 100


#############################################
################### smiles ##################
#############################################
# 读取所有的smiles
smiles = con_function.read_smiles()
# corpus: 索引号     vocab：所有元素
tgt_corpus, tgt_vocab = con_function.load_corpus_smiles(smiles, max_tokens=-1)
# smiles添加特殊符号
smiles_tokens = con_function.new_lines(smiles)
# 填充smiles
tensor_smiles = torch.tensor([con_function.truncate_pad(tgt_vocab[token], tgt_num_steps, tgt_vocab['<pad>']) for token in smiles_tokens])
# print(f'tensor_smiles[0]:  \n\t{tensor_smiles[0]}')

embed_size, num_hiddens, num_layers, dropout = 32, 32, 2, 0.1
# src_num_steps, tgt_num_steps = 100, tgt_num_steps
batch_size, lr, num_epochs, device = 8, 0.005, 1, d2l.try_gpu()

split_ratio = int(len(new_mass_spectras)*0.8)

train_iter, src_vocab, tgt_vocab = con_function.load_data_nmt(
    batch_size=batch_size, source=new_mass_spectras, target=smiles, 
    src_num_steps=src_num_steps, tgt_num_steps=tgt_num_steps)
# train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size=16, num_steps=8)
for X, X_valid_len, Y, Y_valid_len in train_iter:
    # print('X:', X.type(torch.int32))
    print('Y:', Y.type(torch.int32))
    # print('X的有效长度:', X_valid_len)
    # print('Y:', Y.type(torch.int32))
    # print('Y的有效长度:', Y_valid_len)
    break


4253
max_length_mass_spectras: 116
number of len_new_mass_spectras<=100: 4250
percent of total: 99.92946155654832
set(len_new_mass_spectras): {0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 76, 84, 86, 114, 116}
new_mass_spectras_tokens: [191.0, 18.0, 179.0, 99.0, 173.0, 100.0, 155.0, 2.0, 135.0, 10.0]
len(src_corpus): 48466 	len(src_vocab): 5405
0 0 0 0
4253 4253
tensor([191.,  18., 179.,  99., 173., 100., 155.,   2., 135.,  10.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0

In [2]:

import seq_to_seq
from seq_to_seq import Seq2SeqEncoder, Seq2SeqDecoder, train_seq2seq
encoder = Seq2SeqEncoder(len(src_vocab), embed_size, num_hiddens, num_layers, dropout)
decoder = Seq2SeqDecoder(len(tgt_vocab), embed_size, num_hiddens, num_layers, dropout)
net = d2l.EncoderDecoder(encoder, decoder)

# os.environ['CUDA_VISIBLE_DEVICES'] = "4"

# device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu") 
# device = torch.device(['/gpu:4', '/gpu:5'] if torch.cuda.is_available() else "cpu") 


train_seq2seq(net, train_iter, lr, num_epochs, tgt_vocab,  device)




epoch: 0: loss 0.010, 14897.3 tokens/sec on cuda:0


In [10]:

import math
import collections
def bleu(pred_seq, label_seq, k):  #@save
    """计算BLEU"""
    # print(pred_seq, label_seq)
    # pred_tokens, label_tokens = pred_seq.split(' '), label_seq.split(' ')
    # pred_tokens, label_tokens = pred_seq.split(' '), label_seq.split(' ')
    from utils import split
    pred_tokens = split(split(pred_seq)).split()
    label_tokens = split(split(label_seq)).split()
    # print(pred_tokens[:])
    # print(label_tokens[:])
    len_pred, len_label = len(pred_tokens), len(label_tokens)
    score = math.exp(min(0, 1 - len_label / len_pred))
    for n in range(1, k + 1):
        num_matches, label_subs = 0, collections.defaultdict(int)
        for i in range(len_label - n + 1):
            label_subs[' '.join(label_tokens[i: i + n])] += 1
        for i in range(len_pred - n + 1):
            if label_subs[' '.join(pred_tokens[i: i + n])] > 0:
                num_matches += 1
                label_subs[' '.join(pred_tokens[i: i + n])] -= 1
        score *= math.pow(num_matches / (len_pred - n + 1), math.pow(0.5, n))
    return score


In [11]:
src_test = new_mass_spectras[int(len(tensor_mass_spectra)*0.8):]
tgt_test = smiles[int(len(tensor_smiles)*0.8):]

test_iter, src_vocab, tgt_vocab = con_function.load_data_nmt(
    batch_size=batch_size, source=new_mass_spectras[split_ratio:], target=smiles[split_ratio:], 
    src_num_steps=src_num_steps, tgt_num_steps=tgt_num_steps)

for src, tgt in zip(src_test, tgt_test):
    translation, attention_weight_seq = seq_to_seq.predict_seq2seq(net, src, src_vocab, tgt_vocab, src_num_steps, tgt_num_steps, device)
    print(f'{src} => {translation}, bleu {bleu(translation, tgt, k=2):.3f}')


[285.0, 100.0, 255.0, 100.0, 227.0, 100.0] => <bos> <bos> C C ( C O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ), bleu 0.104
[89.0, 59.0, 151.0, 999.0] => <bos> <bos> C C ( C O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ), bleu 0.213
[42.0, 82.0, 43.0, 68.0, 44.0, 999.0, 87.0, 369.0] => <bos> <bos> C C ( C O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ), bleu 0.155
[57.0, 30.0, 69.0, 417.0, 129.0, 194.0, 142.0, 48.0, 146.0, 41.0, 147.0, 561.0, 165.0, 259.0, 182.0, 61.0, 183.0, 999.0] => <bos> <bos> C C ( C O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O ) C ( O 

In [4]:
import os
import torch
from d2l import torch as d2l
import matplotlib.pyplot as plt
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw, MolFromSmiles, MolToSmiles


In [5]:
# mol = Chem.MolFromSmiles('C\\C(\\C=C\\C=C(/C)\\C=C\\[C@H]1C(C)=CCCC1(C)C)=C/C=C/C=C(\\C)/C=C/C=C(\\C)/C=C/C1=C(C)CCCC1(C)C')
# Draw.MolToImage(mol, size=(150,150), kekulize=True)
# # Draw.ShowMol(mol, size=(150,150), kekulize=False)
# Draw.MolToFile(mol, 'data/output.png', size=(600, 600))

mols = []
for s in smiles:
    print(s)
    if s is None:
        continue
    mol = MolFromSmiles(s)
    print(mol)
    Draw.ShowMol(mol, size=(150,150))
    if mol is not None:
        mols.append(mol)
        Draw.ShowMol(mol, size=(150,150), kekulize=False)
# img = Draw.MolsToGridImage(mols, molsPerRow=2, subImgSize=(300,150), kekulize=True)

# Draw.ShowMol(mol, size=(150,150), kekulize=False)
# Draw.MolToFile(mol, 'data/output.png', size=(600, 600))

# from matplotlib.colors import ColorConverter
# img = Draw.MolToImage(mols, highlightAtoms=[1,2], highlightColor=ColorConverter().to_rgb('aqua'))
# img.save('molecule.png')

TclError: no display name and no $DISPLAY environment variable

In [ ]:

import torch.utils.data as Data
from pad import Collate

train_dataset = (new_mass_spectras, smiles)

train_data = Data.DataLoader(dataset=train_dataset, batch_size=32, num_workers=2, collate_fn=Collate())


In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# from con_function import truncate_pad
# x是填充过后的batch数据，seq_lengths是每个样本的序列长度
# seq_lengths = 116

# new_mass_spectras = torch.stack(new_mass_spectras, 0)
# packed_input = pack_padded_sequence(torch.Tensor(new_mass_spectras), seq_lengths, batch_first=True)

src_vocab_mass_spectras = d2l.Vocab(new_mass_spectras, min_freq=0, 
                      reserved_tokens=['<pad>'])
print(len(src_vocab_mass_spectras), src_vocab_mass_spectras)

from build_vocab import Vocab
vocab_mass_spectras = Vocab(new_mass_spectras)
# print(f"token_to_idx: {list(src_vocab_mass_spectras.token_to_idx.items())[:]}")
# print(f"vocab.token_freqs[:10]: \n\t{vocab_mass_spectras.token_freqs[:10]}")

def truncate_pad(line, num_steps, padding_token):
    """截断或填充文本序列"""
    print(line)
    print(line, len(line))
    if len(line) > num_steps:
        return line[:num_steps]
    return line + [padding_token]*(num_steps - len(line))

pad_new_mass_spectras = []
for i in range(len(new_mass_spectras)):
    # print(truncate_pad(src_vocab_mass_spectras[new_mass_spectras[i]], 116, vocab_mass_spectras['<pad>']))
    pad_new_mass_spectra = truncate_pad(new_mass_spectras[i], max(length), padding_token=0)
    pad_new_mass_spectras.append(pad_new_mass_spectra)

# print(pad_new_mass_spectras)

In [ ]:
from utils import split

from rdkit import Chem
from rdkit.Chem import Draw
mol = Chem.MolFromSmiles('CS(=O)CCCCCCCN=C=S')
mol = Chem.MolFromSmiles('Coc1cc(CCN)c(OC)cc1Br')
# Draw.MolToImage(mol, size=(150,150), kekulize=True)

# Draw.ShowMol(mol, size=(150,150), kekulize=False)
Draw.MolToFile(mol, 'data/output.png', size=(600, 600))

def test_split():
    sm = 'C(=O)CC(Br)C[N+]CN'
    pred = 'C ( = O ) C C ( Br ) C [ N + ] C N'
    assert split(sm)==pred
    print(sm)
    print(pred)
    print(split(sm)==pred)
    print(split("Coc1cc(CCN)c(OC)cc1Br"))

test_split()

In [ ]:
from rdkit.Chem import Draw
mol = Chem.MolFromSmiles('COc1cc(CCN)c(OC)cc1Br')
Draw.MolToImage(mol, size=(300,300), kekulize=True)

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

smiles = 'Coc1cc(CCN)c(OC)cc1Br'
mol = Chem.MolFromSmiles(smiles)
# Draw.MolToImage(mol, size=(150,150), kekulize=True)
Draw.ShowMol(mol, size=(150,150), kekulize=False)
# Draw.MolToFile(mol, 'data/output.png', size=(600, 600))

In [ ]:
from rdkit.Chem import Draw
from rdkit import Chem
from rdkit.Chem import AllChem
smis=[
    'COC1=C(C=CC(=C1)NS(=O)(=O)C)C2=CN=CN3C2=CC=C3',
#     'CCN(CC1=C(C=CC(=C1)C(F)(F)F)C2=CC(=C3N2C=NC=C3)CC(=O)O)C(=O)C4CC4',
    'C1=CC2=C(C(=C1)C3=CN=CN4C3=CC=C4)ON=C2C5=CC=C(C=C5)F',
    'COC(=O)C1=CC2=CC=CN2C=N1',
    'C1=C2C=C(N=CN2C(=C1)Cl)C(=O)O',
]
template = Chem.MolFromSmiles('c1nccc2n1ccc2')
AllChem.Compute2DCoords(template)
mols=[]
for smi in smis:
    mol = Chem.MolFromSmiles(smi)
    AllChem.GenerateDepictionMatching2DStructure(mol,template)
    mols.append(mol)
img=Draw.MolsToGridImage(mols,molsPerRow=4,subImgSize=(200,200),legends=['' for x in mols])
img

In [ ]:
# from con_function import new_lines_ms, load_corpus_ms, load_corpus_smiles, new_lines, truncate_pad, load_data_nmt
# from con_function import load_data_nmt
import con_function

C(=O)CC(Br)C[N+]CN
C ( = O ) C C ( Br ) C [ N + ] C N
True
C o c 1 c c ( C C N ) c ( O C ) c c 1 Br
O [ C @ @ H ] 1 C [ C @ @ ] ( O ) ( C [ C @ @ H ] ( O ) [ C @ H ] 1 O C ( = O ) \ C = C \ C 1 = C C ( O ) = C ( O ) C = C 1 ) C ( O ) = O


In [ ]:
# Split SMILES into words
def split(sm):
    '''
    function: Split SMILES into words. Care for Cl, Br, Si, Se, Na etc.
    input: A SMILES
    output: A string with space between words
    '''
    arr = []
    i = 0
    while i < len(sm)-1:
        if not sm[i] in ['%', 'C', 'B', 'S', 'N', 'R', 'X', 'L', 'A', 'M', \
                        'T', 'Z', 's', 't', 'H', '+', '-', 'K', 'F', '@']:
            arr.append(sm[i])
            i += 1
        elif sm[i]=='%':
            arr.append(sm[i:i+3])
            i += 3
        elif sm[i]=='@' and sm[i+1]=='@':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='C' and sm[i+1]=='l':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='C' and sm[i+1]=='a':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='C' and sm[i+1]=='u':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='B' and sm[i+1]=='r':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='B' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='B' and sm[i+1]=='a':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='B' and sm[i+1]=='i':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='S' and sm[i+1]=='i':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='S' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='S' and sm[i+1]=='r':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='N' and sm[i+1]=='a':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='N' and sm[i+1]=='i':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='R' and sm[i+1]=='b':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='R' and sm[i+1]=='a':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='X' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='L' and sm[i+1]=='i':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='A' and sm[i+1]=='l':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='A' and sm[i+1]=='s':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='A' and sm[i+1]=='g':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='A' and sm[i+1]=='u':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='M' and sm[i+1]=='g':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='M' and sm[i+1]=='n':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='T' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='Z' and sm[i+1]=='n':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='s' and sm[i+1]=='i':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='s' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='t' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='H' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='+' and sm[i+1]=='2':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='+' and sm[i+1]=='3':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='+' and sm[i+1]=='4':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='-' and sm[i+1]=='2':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='-' and sm[i+1]=='3':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='-' and sm[i+1]=='4':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='K' and sm[i+1]=='r':
            arr.append(sm[i:i+2])
            i += 2
        elif sm[i]=='F' and sm[i+1]=='e':
            arr.append(sm[i:i+2])
            i += 2
        else:
            arr.append(sm[i])
            i += 1
    if i == len(sm)-1:
        arr.append(sm[i])
    return ' '.join(arr) 


# print(split("O[C@@H]1C[C@@](O)(C[C@@H](O)[C@H]1OC(=O)\C=C\C1=CC(O)=C(O)C=C1)C(O)=O"))
print(split("Coc1cc(CCN)c(OC)cc1Br"))

C o c 1 c c ( C C N ) c ( O C ) c c 1 Br


In [ ]:

# from utils import split
def test_split():
    sm = 'C(=O)CC(Br)C[N+]CN'
    pred = 'C ( = O ) C C ( Br ) C [ N + ] C N'
    assert split(sm)==pred
    print(sm)
    print(pred)
    print(split(sm)==pred)
    print(split("Coc1cc(CCN)c(OC)cc1Br"))

test_split()

print(split("O[C@@H]1C[C@@](O)(C[C@@H](O)[C@H]1OC(=O)\C=C\C1=CC(O)=C(O)C=C1)C(O)=O"))


C(=O)CC(Br)C[N+]CN
C ( = O ) C C ( Br ) C [ N + ] C N
True
C o c 1 c c ( C C N ) c ( O C ) c c 1 Br
O [ C @@ H ] 1 C [ C @@ ] ( O ) ( C [ C @@ H ] ( O ) [ C @ H ] 1 O C ( = O ) \ C = C \ C 1 = C C ( O ) = C ( O ) C = C 1 ) C ( O ) = O
